# Naive Bayes Classifier

Intent classification system based on the multinomial Naive Bayes model for the restaurant information search.






## Required libraries

In [ ]:
# Keras
from tensorflow.keras.preprocessing.text import Tokenizer

# NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# SKlearn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# Joblib
from joblib import dump, load

# NumPy
import numpy as np

# Standard
import pickle

## Paths

The datasets and dictionaries used in this notebook can be downloaded from this [link](https://drive.google.com/drive/folders/178Sv30P-OvoJc_QOOAYkmUHgcQsPg-rb?usp=sharing). Also, it will be necessary to change the path of the `dataset_path` variable to the path wherein the downloaded information is saved. 

**Notes:** 

* The downloaded data can be obtained using the *TextAnalysis_DSTC2.ipynb* notebook.
* Only a UC3M user can access to the download link.

In [ ]:
dataset_path = "/content/drive/My Drive/_TFM/Notebooks/Datasets/"
naive_path = "/content/drive/My Drive/_TFM/Notebooks/Models/naive/"

# Download some tools for the NLTK library
nltk.download("wordnet")
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Data reading

DSTC 2 dataset provides dialogues of a human talking to a information system labelled with slots and dialogue actions.

In our case, the labels will be the users' intents which are the combination of acts and slots.

The data only has information about users' utterances.

**Load dataset**

In [ ]:
# x_train
x_train_obj = open(dataset_path + "x_train.obj", "rb")
x_train = pickle.load(x_train_obj)
x_train_obj.close()

# y_train
y_train_obj = open(dataset_path + "y_train.obj", "rb")
y_train = pickle.load(y_train_obj)
y_train_obj.close()

# x_valid
x_valid_obj = open(dataset_path + "x_valid.obj", "rb")
x_valid = pickle.load(x_valid_obj)
x_valid_obj.close()

# y_valid
y_valid_obj = open(dataset_path + "y_valid.obj", "rb")
y_valid = pickle.load(y_valid_obj)
y_valid_obj.close()

# x_test
x_test_obj = open(dataset_path + "x_test.obj", "rb")
x_test = pickle.load(x_test_obj)
x_test_obj.close()

# y_test
y_test_obj = open(dataset_path + "y_test.obj", "rb")
y_test = pickle.load(y_test_obj)
y_test_obj.close()

**Check data**

In [ ]:
print("Data = (X: users' utterances, y: intents)")

print("\n  > Train data:")
print("Samples: 8148 = %s" % len(x_train))
print(list(zip(x_train, y_train))[:3])

print("\n  > Validation data:")
print("Samples: 5656 = %s" % len(x_valid))
print(list(zip(x_valid, y_valid))[:3])

print("\n  > Test data:")
print("Samples: 5769 = %s" % len(x_test))
print(list(zip(x_test, y_test))[:3])

Data = (X: users' utterances, y: intents)

  > Train data:
Samples: 8148 = 8148
[('cheap restaurant', ['inform_pricerange']), ('any', ['inform_this']), ('south', ['inform_area'])]

  > Validation data:
Samples: 5656 = 5656
[('sil', ['unknown']), ('north part of town serving gastropub food', ['inform_food', 'inform_area']), ('north part of town serving gastropub food', ['inform_food', 'inform_area'])]

  > Test data:
Samples: 5769 = 5769
[('uh yes im looking for a cheap restaurant in the west part of town', ['affirm', 'inform_pricerange', 'inform_area']), ('west', ['inform_area']), ('uh yes a cheap restaurant', ['affirm', 'inform_pricerange'])]


## Data preprocessing


Each text sample will follow the next pipeline:

*raw text* > **decontraction** > **normalization** > **lemmatization** > **lowercasing** > **tokenization** > *tokens*

And then, we will build the vocabulary including the UNK token.

**Settings**

In [ ]:
# Merge all samples
x_data = np.concatenate([x_train, x_valid, x_test])

In [ ]:
# Decontract funtion
contracted_words_obj = open(dataset_path + "contracted_words.obj", "rb")
contracted_words = pickle.load(contracted_words_obj)
contracted_words_obj.close()

def decontract(text):
  for word in text.split():
    if word.lower() in contracted_words:
      text = text.replace(word, contracted_words[word.lower()])
  return text

# Normalize function
normalized_words_obj = open(dataset_path + "normalized_words.obj", "rb")
normalized_words = pickle.load(normalized_words_obj)
normalized_words_obj.close()

def normalize(text):
  for word in text.split():
    if word.lower() in normalized_words:
      text = text.replace(word, normalized_words[word.lower()])
  return text

In [ ]:
# Lemmatize function

# Set the lemmatizer
lemmatizer = WordNetLemmatizer()

# Get the POS tag for a given word
def get_pos_tag(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

# Lemmatize a text with POS tag
def lemmatize(text, lemmatizer):
  word_list = nltk.word_tokenize(text)
  text_lemmatized = ' '.join([lemmatizer.lemmatize(word, get_pos_tag(word)) for word in word_list])
  return text_lemmatized

**Preprocessing**

In [ ]:
# Decontract, normalize, lemmatize, lowercase and tokenize the text samples
tokenizer = Tokenizer(lower=True, oov_token="unk")
tokenizer.fit_on_texts(
    [lemmatize(normalize(decontract(sample)), lemmatizer) for sample in x_data])

# Vocabulary
x_vocab = tokenizer.word_index
x_vocab_size = len(x_vocab) + 1

In [ ]:
x_vocab_size

502

## Featurization

### Tokens (Word Counts)


In [ ]:
# x_train
x_train_eval = np.concatenate([x_train, x_valid])
x_train_eval_wc = tokenizer.texts_to_matrix(
    [lemmatize(normalize(decontract(sample)), lemmatizer) for sample in x_train_eval], mode="count")

# x_test
x_test_wc = tokenizer.texts_to_matrix(
    [lemmatize(normalize(decontract(sample)), lemmatizer) for sample in x_test], mode="count")

In [ ]:
np.shape(x_train_eval_wc)

(13804, 502)

In [ ]:
np.shape(x_test_wc)

(5769, 502)

### Labels (Multi-Label Binarizer)

On the other hand, the labels will be encoded as *n* binary elements in an array, where *n* is the total number of labels.

The binary vector (associated to each sample) indicates the presence of labels.

In [ ]:
# Load the labels
labels_obj = open(dataset_path + "labels.obj", "rb")
labels = pickle.load(labels_obj)
labels_obj.close()

In [ ]:
# Build the multi-label binarizer
mlb = MultiLabelBinarizer(classes=labels)

# y_train
y_train_eval = np.concatenate([y_train, y_valid])
y_train_eval_mlb = mlb.fit_transform(y_train_eval)

# y_test
y_test_mlb = mlb.fit_transform(y_test)

## Multinomial Naive model

In a multi-label scenario the Multinomial Naive Bayes model must be wrapped in the OneVsRestClassifier to apply the One-vs-the-Rest (OvR) multi-label strategy.

### Hyper-parameter tuning

In [ ]:
# Define the model
naive_model = OneVsRestClassifier(MultinomialNB(), n_jobs=1)

naive_model.get_params().keys()

dict_keys(['estimator__alpha', 'estimator__class_prior', 'estimator__fit_prior', 'estimator', 'n_jobs'])

In [ ]:
# Define the index of the train and validation data
index_data = [i for i, _ in enumerate(x_train_eval)]
index_train = index_data[:len(x_train)]
index_valid = index_data[len(x_train):]

In [ ]:
# Define the hyper-parameters to tuning
alpha = [1e-03, 1.0, 1.5, 2.0, 2.5, 3.0]
param_grid = dict(estimator__alpha=alpha)

In [ ]:
# Tuning through Grid Search approach
naive_tune_model = GridSearchCV(naive_model,
                                param_grid=param_grid,
                                scoring = "accuracy",
                                cv=[(index_train, index_valid)],
                                verbose=1,
                                n_jobs=-1)

In [ ]:
# Select the model with the best hyper-parameters
naive_tune_model = naive_tune_model.fit(x_train_eval_wc, y_train_eval_mlb)

Fitting 1 folds for each of 6 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    5.1s finished
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 27 is present in all training examples.
  str(classes[c]))


In [ ]:
print("--- Hyper-paremeters selected ---\n")
print("Best accuracy: %f using %s" % (naive_tune_model.best_score_, naive_tune_model.best_params_))

--- Hyper-paremeters selected ---

Best accuracy: 0.687412 using {'estimator__alpha': 1.5}


**Save the model**

In [ ]:
dump(naive_tune_model, naive_path + "naive_model.joblib")

['/content/drive/My Drive/_TFM/Notebooks/Models/naive/naive_model.joblib']

**Load the model**

In [ ]:
naive_tune_model = load(naive_path + "naive_model.joblib") 

### Model Evaluation

In [ ]:
# Prediction
y_test_mlb_pred = naive_tune_model.predict(x_test_wc)

**Accuracy**

In [ ]:
metrics.accuracy_score(y_test_mlb, y_test_mlb_pred)

0.718148725949038

**Precision, Recall & F1-Score**

In [ ]:
print(metrics.classification_report(y_test_mlb, y_test_mlb_pred, target_names=labels, zero_division=0))

                    precision    recall  f1-score   support

               ack       0.00      0.00      0.00         7
            affirm       0.87      0.97      0.92       326
               bye       0.94      1.00      0.97       619
      confirm_area       0.00      0.00      0.00         2
      confirm_food       0.17      0.16      0.17        25
confirm_pricerange       0.00      0.00      0.00         7
         deny_food       0.00      0.00      0.00         2
         deny_name       0.00      0.00      0.00         0
             hello       0.29      0.18      0.22        28
       inform_area       0.77      0.95      0.85       653
       inform_food       0.88      0.99      0.93      1439
       inform_name       0.17      0.07      0.10        15
 inform_pricerange       0.58      0.96      0.72       526
       inform_this       0.91      0.96      0.94       537
            negate       0.74      0.84      0.79        69
            repeat       0.50      0.21

### Predictions

We inspect some predictions.

In [ ]:
y_test_pred = mlb.inverse_transform(y_test_mlb_pred)

In [ ]:
index_sample = 0

print("User's utterance: %s" % x_test[index_sample])

print("\nTrue label: %s" % y_test[index_sample])
print("Predicted label: %s" % list(y_test_pred[index_sample]))

print("\nTrue binary label: %s" % y_test_mlb[index_sample])
print("Predicted binary label: %s" % y_test_mlb_pred[index_sample])

User's utterance: uh yes im looking for a cheap restaurant in the west part of town

True label: ['affirm', 'inform_pricerange', 'inform_area']
Predicted label: ['inform_area', 'inform_pricerange']

True binary label: [0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Predicted binary label: [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
